## Pytorch的自动微分算法 auto_grad

在深入了解Pytorch自动微分算法前，需要对pytorch的自动微分算法有一定了解，这样在涉及到反向传播时梯度在多卡之间的通信时才能更加熟练

反向传播的原理在此不再赘述，我们只关注反向传播和自动微分在Pytorch中的实现，且更多关注API。首先导入torch，定义示例向量。注意这里我们指定向量是需要梯度的，即`require_grad=True`

In [1]:
import torch

x_1 = torch.tensor(2.0, requires_grad=True)
b_1 = torch.tensor(1.0, requires_grad=True)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


我们定义一个函数:

$$ y_1 = x_1^3 + b_1 $$

显然，$y_1$对$x_1$的导数为$\frac{y_1}{x_1}=3x^2$，对$b_1$的导数为1。我们利用代码来验证之：

In [2]:
y_1 = x_1 ** 3 + b_1
y_1.backward()
print(y_1, x_1.grad, b_1.grad)

tensor(9., grad_fn=<AddBackward0>) tensor(12.) tensor(1.)


除了直接对输出值$y_1$使用`.backward()`方法，还可以通过`torch.autograd.grad()`函数来进行自动求导。该函数接受输入值为待求导的方程、输入变量，返回值为梯度，**其中梯度的形状和输入变量的形状一模一样**

注意：前面所使用的`.backward()`方法返回值始终为`None`

In [3]:
x_2 = torch.tensor(2.0, requires_grad=True)
b_2 = torch.tensor(1.0, requires_grad=True)
y_2 = x_2 ** 3 + b_2
grads = torch.autograd.grad(y_2, inputs=[x_2, b_2])
print(grads)

(tensor(12.), tensor(1.))


对于高维的输入输出，正常计算导数需要用到雅可比矩阵，而在pytorch内实现则非常方便。此时梯度的形状与自变量的形状一致。例如：

In [4]:
a = torch.tensor([[2.0, 3.0], [4.0, 5.0]], requires_grad=True)
b = torch.tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)  # 2x2
z = a ** 2 + a * b + b ** 2
grads = torch.autograd.grad(z, inputs=[a, b], grad_outputs=torch.ones_like(z))
print(grads)
print(grads[0].shape, grads[1].shape)

(tensor([[ 5.,  8.],
        [11., 14.]]), tensor([[ 4.,  7.],
        [10., 13.]]))
torch.Size([2, 2]) torch.Size([2, 2])


接下来，我们引用pytorch官方tutorial的一个简单的神经网络的例子。考虑最简单的一层神经网络，具有输入`x`、参数`w`和`b`，以及一些损失函数。它可以通过以下方式在 PyTorch 中定义：

In [5]:
x = torch.ones(5)               # input tensor
y = torch.zeros(3)              # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

其中`w`和`b`是我们想要优化的参数，因此我们设置了他们的属性`requires_grad=True`。除此之外，还可以在稍后使用`w.requires_grad_(True)`来设置。

众所周知，神经网络可以看作一张计算图。我们将上述这些矢量构建到计算图内，实际上是创建了一个`Function`类的对象，该对象知道如何在前向计算函数，以及如何在反向传播步骤中计算其导数。对反向传播函数的引用存储在`grad_fn`张量的属性中。

In [6]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f53d04e7e50>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f53d04e7f40>


在神经网络的训练过程中，我们希望能从网络的前向输出计算梯度，从而对参数`w`和`b`进行更新。我们采用最简单的`.backward()`方法，计算并观测这些参数的梯度：

In [7]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0255, 0.0777, 0.0249],
        [0.0255, 0.0777, 0.0249],
        [0.0255, 0.0777, 0.0249],
        [0.0255, 0.0777, 0.0249],
        [0.0255, 0.0777, 0.0249]])
tensor([0.0255, 0.0777, 0.0249])


这里有两点需要注意：

1. 只有设置为`requires_grad=True`的参数（或言：计算图中的叶子节点）才能得到它们的`grad`属性，其他的节点没法得到梯度；

2. 正常情况下，为了优化性能，一次前向、后向计算完毕后，该计算图就会被销毁。如果我们需要在同一个图上多次调用backward，则需要在backward调用时候设置 `retain_graph=True`。

在神经网络的验证valid/测试test过程中，为了加快进度，我们不需要也不必计算梯度，此时我们可以利用pytorch提供的`torch.no_grad()`函数来实现：

In [8]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


使用了`with torch.no_grad():`环境后，所有梯度计算都会禁止梯度的计算。实现相同结果的另一种方法是在张量上使用`detach()`方法：

In [9]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


## 大神经网络的例子

一般来说常用的代码库都会将神经网络利用`torch.nn.Module`来进行封装。接下来演示在这种更常见的情况下，如何访问和操作模型参数的梯度

In [10]:
from torch import nn

# 一个简单的MLP
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(8, 12)  # Hidden layer
        self.act = nn.ReLU()
        self.output = nn.Linear(12, 4)  # Output layer

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)
 
# 实例化MLP类得到net
net = MLP()

# 访问net的第一个隐藏层的权重和偏差
w1 = net.hidden.weight
b1 = net.hidden.bias
print(w1.shape, b1.shape) 

torch.Size([12, 8]) torch.Size([12])


注意这里采用的是`net.<name>.weight`的方式来访问权重的。如果不清楚模型中每一层的名字，可以采用以下方法：

In [11]:
for name, param in net.named_parameters():
    print(name, param.shape)

hidden.weight torch.Size([12, 8])
hidden.bias torch.Size([12])
output.weight torch.Size([4, 12])
output.bias torch.Size([4])


在此基础上，访问这些权重的梯度只需要多索引一次`.grad`即可：

In [12]:
# 访问net的第一个隐藏层的权重和偏差的梯度
w1_grad = net.hidden.weight.grad 
b1_grad = net.hidden.bias.grad
print(w1_grad, b1_grad)

None None


注意这里的输出虽然为`None`，但这些层的`require_grad`在`nn.Module`内是默认为`True`的，输出为空只是因为网络刚被初始化：

In [13]:
print(net.hidden.weight.requires_grad)
print(net.hidden.bias.requires_grad)

True
True


我们假装对该网络做一个训练，可以看到网络的参数的梯度被计算，进而被应用来更新参数：

In [14]:
# 做一个假的数据集，30个样本，每个样本8维特征，4维标签
data = torch.randn(30, 8)
label = torch.randn(30, 4)

dataset = torch.utils.data.TensorDataset(data, label)
dataloder = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

# 开始训练
net.train()
net.cuda()      # 可选，如果有GPU的话

# 我们只训练一个epoch即可
for index, (data, label) in enumerate(dataloder):
    data = data.cuda()
    label = label.cuda()
    out = net(data)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(out, label)
    loss.backward()
    print(f"step: {index}, loss: {loss}, weight grad for hidden layer: {net.hidden.weight.grad}")

step: 0, loss: 0.7470499873161316, weight grad for hidden layer: tensor([[ 1.5022e-02, -3.0880e-03,  3.0836e-03, -1.2889e-02, -3.2447e-02,
          4.8631e-03,  7.8574e-03, -4.6769e-03],
        [ 1.2990e-02, -9.3525e-03, -1.7760e-02,  4.2679e-03, -1.1164e-02,
         -2.3242e-03, -5.0472e-03,  2.7768e-03],
        [-1.3842e-02,  1.3202e-02,  1.6635e-02, -9.7227e-03,  3.8987e-03,
         -6.7535e-03,  3.0358e-03,  3.1507e-03],
        [ 1.3816e-02, -7.6427e-03, -1.2734e-03,  5.0467e-04, -7.3625e-03,
          2.8755e-03, -4.0851e-03,  3.6573e-03],
        [-2.5245e-02,  1.2688e-02, -2.0006e-02,  8.2256e-02,  3.5192e-02,
          1.9001e-03, -9.4666e-04, -3.8687e-02],
        [-3.8535e-03,  5.7771e-03, -1.2471e-02,  3.4457e-02,  1.4222e-02,
          5.2874e-03, -2.0068e-03, -2.3625e-02],
        [-2.4186e-02,  3.9408e-03, -1.4809e-02, -1.6549e-03,  6.2134e-03,
         -1.1559e-02, -6.6432e-03, -5.0576e-04],
        [-3.1156e-02,  2.5198e-02, -2.7281e-03,  6.1452e-02,  5.2364e-02,


可见在训练阶段，模型的参数的梯度被计算，进而用于更新模型参数